# 控制流程

- 以區為單位切割

- 再以頁為單位做切割 取出連結

- 輸入區搜尋=> 翻十頁 或 翻到我過去抓到的檔案最新的資料時停下 開始抓取連結 =>取得連結後開始抓取文章內容 =>存成csv檔

- 為了避免中間抓取文章失敗 當失敗時丟exception 先存檔目前進度 之後再讀檔判斷 回到上一個步驟

In [ ]:
#匯入所需套件
import requests
from bs4 import BeautifulSoup as bs
import pandas
import time

In [ ]:
# TOR 用
# import requesocks
# URL  = 'http://udn.com/news/story/1/1252625'
# session = requesocks.session()
# session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
# res = session.get(URL)
# print(res.status_code)
# print(res.headers['content-type'])
# print(res.text)

In [146]:
#儲存文章連結 正規表示法吃連結 用來建立連結資料
def getlinkv2 (script):
    global count
    m = re.findall('(http://udn.com/news/.*\d+)',script)
    for link in m:
        links = {'link':link,'area':area,'page':page,'count':count}
        linkary.append(links)
        count +=1

In [147]:
#requests 方法 找出網頁特性 發現是jsp.net 直接對伺服器發送要求
import requests
import re
import time
#列出所需查詢區域 一開始先測試3區的效果
#['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':''}
linkary =[]
#每一區
for area in ary:
    count = 1
    headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ area,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
#每一頁    
    for page in range(1,11):
        res =requests.get('http://search.udn.com/search/searchResult4utf8.jsp?ch=udn.news2014&df=2&rc=15&wc=80&pw=220&mc=&q=' + \
                          area + '&fp={}'.format(page),headers = headers,timeout=3)
        time.sleep(3) #避免讓伺服器察覺攻擊
        script = res.text
        getlinkv2(script)
    

In [149]:
import pandas as pd
data1 = pd.DataFrame(linkary)

In [153]:
#輸出成CSV檔將連結存起 假如需要可以讀回
data1.to_csv('link.csv',encode='utf-8')
#讀回程式
import csv
linkary = []
with open('link.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

- 一般版

In [ ]:
#取得文章內容及標題
import requests
from bs4 import BeautifulSoup as bs
import pandas
import time

for get in linkary:
    #強制取得 建議不要使用while 在一般requests套件中 會被伺服器端察覺
    while True:
        try:
            url = get['link']
            res = requests.get(url,headers=headers)
            soup = bs(res.content)
            break
        except:
            print 'try again 5sec'
            time.sleep(5)
    for article in soup.select('#story_body_content'):
        try:
            get['title'] = article.select('h2')[0].text.strip()
            get['content'] = article.select('p')[0].text.strip()
            get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
            
            #get['time'] = article.select('h3')[0].text.strip().split(' ')[1] 發文時間可要可不要
        except:
            print 'this link has some problem'
            print get['link'], get['area'], get['count']
            continue

In [ ]:
#輸出完成品
import pandas as pd
data2 = pd.DataFrame(linkary)
data2.to_csv('ok.csv',encode='utf-8')

- tor版 請勿在windows 沒有tor socket 底下使用

In [172]:
#tor time to gentle haha
import requesocks
from bs4 import BeautifulSoup as bs
import pandas
import time

session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
for get in linkary:
    while True:
        try:
            url = get['link']
            res = session.get(url,headers=headers)
            break
        except:
            print 'try again 5sec'
            time.sleep(5)
    soup = bs(res.content)
    for article in soup.select('#story_body_content'):
        try:
            get['title'] = article.select('h2')[0].text.strip()
            get['content'] = article.select('p')[0].text.strip()
            get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
            get['time'] = article.select('h3')[0].text.strip().split(' ')[1]
        except:
            print 'this link has some problem'
            print get['link'], get['area'], get['count']
            continue #找不到就做下一筆

{'count': 1, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/6655/1270657', 'page': 1}
{'count': 2, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/7741/1270406', 'page': 1}
{'count': 3, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/7210/1266639', 'page': 1}
{'count': 4, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/6/1262798', 'page': 1}
{'count': 5, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/6/1262782', 'page': 1}
{'count': 6, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'content': 'content', 'hi': 'hi', 'link': u'http://udn.com/news/story/6/1262724', 'page': 1}
{'count': 7, 'area': '\xe4\xb8\xad\xe6\xad\xa3\xe5\x8d\x80', 'conte

In [ ]:
import pandas as pd
data2 = pd.DataFrame(linkary)
data2.to_csv('oktor.csv',encode='utf-8')